<a href="https://colab.research.google.com/drive/1kxqf1JrDgmgY-rdwuZ2klYw2LILZJesz?usp=sharing">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>


In [1]:
#1 Install required libraries
!pip install -q unsloth "torch>=2.1" transformers peft datasets accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.2/193.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
# Step 2: Load TinyLlama model and extend context
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    max_seq_length=8192,    # Extended to 8K tokens
    dtype=torch.float16,
    load_in_4bit=True,
)

tokenizer.pad_token = tokenizer.eos_token

==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: unsloth/tinyllama-chat-bnb-4bit can only handle sequence lengths of at most 2048.
But with kaiokendev's RoPE scaling of 4.0, it can be magically be extended to 8192!


In [30]:
# Step 2: Load TinyLlama model and extend context
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    max_seq_length=4096,
    dtype=torch.float16,
    load_in_4bit=True,
)

tokenizer.pad_token = tokenizer.eos_token

==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [31]:
#4 Confirm the tokenizer and model configurations
print(f"Model context size (max_position_embeddings): {model.config.max_position_embeddings}")
print(f"Tokenizer max length: {tokenizer.model_max_length}")

Model context size (max_position_embeddings): 4096
Tokenizer max length: 4096


In [32]:
# Step 5: Save model and tokenizer to local folder
save_path = "tinyllama_context"

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('tinyllama_context/tokenizer_config.json',
 'tinyllama_context/special_tokens_map.json',
 'tinyllama_context/tokenizer.model',
 'tinyllama_context/added_tokens.json',
 'tinyllama_context/tokenizer.json')

In [33]:
# Step 7: Reload model to verify it supports 8K context
from transformers import AutoModelForCausalLM, AutoTokenizer

reloaded_model = AutoModelForCausalLM.from_pretrained(
    save_path,
    device_map="auto",
    torch_dtype=torch.float16,
)
reloaded_tokenizer = AutoTokenizer.from_pretrained(save_path)

print(f"Reloaded model maximum context size: {reloaded_model.config.max_position_embeddings}")

Reloaded model maximum context size: 4096


In [42]:
# Step 8: Generate output to test model

prompt = "In the kingdom of Eldoria, nestled between shimmering mountains and vast oceans, there existed a library that held the secrets of ancient civilizations. Scholars from around the world journeyed there, seeking wisdom lost to time. The scrolls and manuscripts, written in forgotten languages, spoke of heroes, of fallen kingdoms, of inventions that defied imagination. The guardians of the library, wise and mysterious, allowed only the most deserving to unlock these treasures" * 20  # Medium-long prompt

inputs = reloaded_tokenizer(prompt, return_tensors="pt", truncation=True).to("cuda")

with torch.no_grad():
    outputs = reloaded_model.generate(
        **inputs,
        max_new_tokens=50,   # Safer for memory
        temperature=0.9,
        top_p=0.9,
        pad_token_id=reloaded_tokenizer.eos_token_id,
    )

print("\n--- Output Generated ---\n")
print(reloaded_tokenizer.decode(outputs[0], skip_special_tokens=True))


--- Output Generated ---

In the kingdom of Eldoria, nestled between shimmering mountains and vast oceans, there existed a library that held the secrets of ancient civilizations. Scholars from around the world journeyed there, seeking wisdom lost to time. The scrolls and manuscripts, written in forgotten languages, spoke of heroes, of fallen kingdoms, of inventions that defied imagination. The guardians of the library, wise and mysterious, allowed only the most deserving to unlock these treasuresIn the kingdom of Eldoria, nestled between shimmering mountains and vast oceans, there existed a library that held the secrets of ancient civilizations. Scholars from around the world journeyed there, seeking wisdom lost to time. The scrolls and manuscripts, written in forgotten languages, spoke of heroes, of fallen kingdoms, of inventions that defied imagination. The guardians of the library, wise and mysterious, allowed only the most deserving to unlock these treasuresIn the kingdom of Eldor